# Week 14：檢索增強生成（RAG）、工具呼叫與結構化輸出

你好！在本週的實驗課程中，我們將探索一些強大的技術，來擴展大型語言模型（LLM）的功能。我們將會學到：

1.  **嵌入（Embeddings）**:
    * 了解如何使用 `sentence-transformers` 和 OpenAI API 來將文字轉換為向量表示（嵌入）。
    * 計算文字之間的相似度。
2.  **向量資料庫（Vector Databases）**:
    * 使用 LanceDB 來儲存和檢索我們產生的嵌入。
    * 了解如何有效地查詢向量資料庫以找到相關資訊。
3.  **檢索增強生成（Retrieval Augmented Generation - RAG）**:
    * 結合向量資料庫的檢索能力與 LLM 的生成能力，讓模型可以根據外部知識回答問題。
4.  **工具呼叫（Function Calling/Tool Calling）**:
    * 學習如何讓 LLM 呼叫外部工具或函式（例如我們建立的電影資料庫查詢工具）。
    * 了解如何定義工具的結構描述（schema），以及處理模型的工具呼叫請求。
5.  **結構化輸出（Structured Outputs）**:
    * 學習如何讓 LLM 以特定的 JSON 格式輸出結果，確保輸出的可預測性和一致性。

我們將會使用電影資料集作為範例，一步一步地完成這些概念的實作。準備好了嗎？開始囉！

In [ ]:
!git clone https://github.com/dachenlian/genai4humanities-wk14.git wk14
!rm wk14/week14.ipynb
!mv wk14/* .
!curl -LsSf https://astral.sh/uv/install.sh | sh
!uv pip install --system -r pyproject.toml

# 匯入（Imports）

首先，我們需要匯入所有必要的 Python 套件。
`autoreload` 可以讓我們在修改外部 Python 檔案後，不用重新啟動 Jupyter核心就能自動載入更新。

In [ ]:
%load_ext autoreload
%autoreload 2

import asyncio
import datetime
import json
import os
from enum import StrEnum
from typing import Annotated  # 用於更精確的型別提示

import lancedb  # 向量資料庫
import pandas as pd  # 資料處理
import tiktoken  # OpenAI 的 token 計算工具
import torch  # PyTorch，用於張量運算
import torch.nn.functional as F  # PyTorch 的函式庫，用於計算餘弦相似度等
from datasets import load_dataset, load_from_disk  # Hugging Face Datasets 套件
from dotenv import load_dotenv  # 讀取環境變數
from huggingface_hub import AsyncInferenceClient  # Hugging Face 推論 API 客戶端
from huggingface_hub.inference._generated.types import ChatCompletionOutputToolCall
from loguru import logger  # 更方便的日誌記錄
from pydantic import BaseModel, Field  # 資料驗證與設定管理
from openai import OpenAI, AsyncOpenAI  # OpenAI API 客戶端
from openai.types.chat.chat_completion_message_tool_call import (
    ChatCompletionMessageToolCall,
)
from sentence_transformers import SentenceTransformer  # 句子嵌入模型
from tenacity import (  # 用於重試機制
    RetryCallState,
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

from utils import create_tool_schema_for_function  # 自訂的工具函式
from tool_types import ToolCallResult  # 自訂的型別

load_dotenv()  # 載入 .env 檔案中的環境變數

try:
    from google.colab import userdata  # type: ignore # 嘗試從 Google Colab 讀取秘密金鑰
except ImportError:
    userdata = None

load_dotenv(override=True)  # 再次載入，確保 .env 的設定會覆寫已存在的環境變數
pd.set_option(
    "display.max_colwidth", 0
)  # 設定 pandas DataFrame 顯示欄位的最大寬度（0 代表不限制）

# 準備資料集（Preparing dataset）

我們將使用 [TMDB 5000 Movies](https://huggingface.co/datasets/AiresPucrs/tmdb-5000-movies) 資料集。
這個資料集包含了約 5000 部電影的資訊，例如：概覽（overview）、類型（genres）、關鍵字（keywords）等。

In [ ]:
# https://huggingface.co/datasets/AiresPucrs/tmdb-5000-movies
ds = load_dataset("AiresPucrs/tmdb-5000-movies", split="train")
ds

## 資料預處理
我們需要對資料集進行一些清理和轉換。

In [ ]:
# 移除 "overview" 欄位為空值的電影
ds = ds.filter(lambda x: bool(x["overview"]))
ds

In [ ]:
def preprocess(example: dict) -> dict:
    """
    對單筆電影資料進行預處理。
    - 將 JSON 字串解析為 Python 列表或字典。
    - 轉換日期格式。
    - 提取年份。
    """
    example["genres"] = [g["name"] for g in json.loads(example["genres"])]
    example["keywords"] = [k["name"] for k in json.loads(example["keywords"])]
    example["release_date"] = datetime.datetime.strptime(
        d if (d := example["release_date"]) else "1970-01-01", "%Y-%m-%d"
    ).date()
    example["release_year"] = example["release_date"].year
    example["spoken_languages"] = [
        sl["name"] for sl in json.loads(example["spoken_languages"])
    ]
    example["cast"] = [
        {
            "name": c["name"],
            "character": c["character"],
        }
        for c in json.loads(example["cast"])
    ]
    # 以下是被註解掉的欄位，暫不處理
    # example["production_companies"] = [
    #     pc["name"] for pc in json.loads(example["production_companies"])
    # ]
    # example["production_countries"] = [
    #     pc["name"] for pc in json.loads(example["production_countries"])
    # ]
    # example["crew"] = [
    #     {
    #         "name": c["name"],
    #         "job": c["job"],
    #     }
    #     for c in json.loads(example["crew"])
    # ]
    return example

In [ ]:
# 使用 .map() 方法對整個資料集進行預處理
# remove_columns 會移除指定的原始欄位
# num_proc 設定了並行處理的程序數量，可以加速處理
ds = ds.map(
    preprocess,
    remove_columns=[
        "id",  # 電影ID，通常不需要作為特徵
        "homepage",  # 電影主頁連結
        "production_companies",  # 製片公司
        "production_countries",  # 製片國家
        "status",  # 發行狀態
        "tagline",  # 電影標語
        "vote_count",  # 投票數量
        "vote_average",  # 平均評分
        "crew",  # 工作人員
        "original_title",  # 原始標題 (稍後會用 'title'，若要保留原始標題可取消註解)
    ],
    num_proc=4,
)
ds[0]  # 顯示第一筆處理後的資料

# 嵌入（Embeddings）

嵌入是將文字轉換為數值向量的過程。這些向量可以捕捉文字的語義資訊，讓機器可以理解文字之間的關係。
意思相近的文字，其向量在向量空間中的距離也會比較近。

我們會比較兩種產生嵌入的方式：
1.  **sentence-transformers**: 一個開源的 Python 套件，提供了許多預訓練好的模型，可以在本機端執行。
2.  **OpenAI API**: OpenAI 提供的付費服務，可以透過 API 取得高品質的文字嵌入。

In [ ]:
queries = [
    "I want to watch an exciting superhero movie",  # 我想看一部刺激的超級英雄電影
    "我想看一部超級英雄電影",  # 中文查詢
]

movies = [
    "A movie about a group of friends who go on a road trip",  # 一部關於一群朋友公路旅行的電影
    "A romantic comedy about a couple who meet at a wedding",  # 一部關於一對在婚禮上相遇的情侶的浪漫喜劇
    "An autobiography of George Washington, the first president of the United States",  # 美國第一任總統喬治華盛頓的自傳
    "Spider-Man is fighting against the Green Goblin in another universe",  # 蜘蛛人在另一個宇宙對抗綠惡魔
]

## sentence-transformers

* [官方文件](https://sbert.net/)
* 支援 `sentence-transformers` 的模型：https://huggingface.co/models?library=sentence-transformers
* 用於 `sentence-similarity`（句子相似度）任務的模型：https://huggingface.co/models?pipeline_tag=sentence-similarity

我們將使用 `paraphrase-multilingual-MiniLM-L12-v2` 模型，它支援多種語言，包含英文和中文。
`device="cpu"` 表示我們將在 CPU 上執行模型。如果有 GPU，可以改為 `"cuda"`。

In [ ]:
embedder = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device="cpu",
)

### 計算嵌入（Compute embeddings）

使用 `embedder.encode()` 方法將文字轉換為嵌入向量。

In [ ]:
query_embeddings = embedder.encode(queries)
movie_embeddings = embedder.encode(movies)
print(
    query_embeddings.shape, movie_embeddings.shape
)  # .shape 顯示向量的維度 (數量, 維度大小)
print(f"Query: {query_embeddings[0][:5]}")  # 顯示第一個查詢向量的前5個維度值
print(f"Movie: {movie_embeddings[0][:5]}")  # 顯示第一個電影描述向量的前5個維度值

### 計算相似度並擷取前 K 個（Computing similarity + retrieving top k）

我們可以使用 `embedder.similarity()` 方法來計算兩組嵌入之間的餘弦相似度。
餘弦相似度的值介於 -1 和 1 之間，越接近 1 代表越相似。

In [ ]:
# 計算查詢嵌入和電影嵌入之間的餘弦相似度
similarities = embedder.similarity(query_embeddings, movie_embeddings)
similarities  # 輸出一個矩陣，列代表電影，欄代表查詢

In [ ]:
YELLOW = "\033[33m"  # ANSI escape code，讓文字顯示為黃色
END = "\033[0m"  # ANSI escape code，重設文字顏色
# 輸出所有查詢與電影的相似度分數
for idx_i, sentence1 in enumerate(queries):
    print(sentence1)
    for idx_j, sentence2 in enumerate(movies):
        print(
            f" - {sentence2: <30}: {YELLOW}{similarities[idx_i][idx_j]:.4f}{END}"
        )  # :.4f 表示格式化為小數點後4位

In [ ]:
# torch.topk() 可以找到張量中最大或最小的 k 個元素及其索引
# similarities[0] 是第一個查詢與所有電影的相似度
torch.topk(similarities[0], k=4)  # k=4 表示找出最相似的4部電影

In [ ]:
# 輸出每個查詢最相似的 k 部電影
for idx_i, sentence1 in enumerate(queries):
    print(sentence1)
    # torch.topk().indices 會回傳前 k 個最相似電影的索引
    for idx_j in torch.topk(similarities[idx_i], k=4).indices:
        print(f" - {movies[idx_j]: <30}: {YELLOW}{similarities[idx_i][idx_j]:.4f}{END}")

## OpenAI

[官方文件](https://platform.openai.com/docs/guides/embeddings?lang=python)

使用 OpenAI 的嵌入模型通常可以得到更好的效果，但需要 API 金鑰且有費用。

我們將使用 `text-embedding-3-small` 模型，它在成本和效能之間取得了不錯的平衡。

![](https://i.redd.it/lpf0u9nbj7w41.jpg)

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key and userdata:
    # 如果在 Google Colab 環境，嘗試從 userdata 取得 API 金鑰
    api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY 環境變數未設定")

# 使用 AsyncOpenAI 進行非同步 API 呼叫，可以提高效率
client = AsyncOpenAI(api_key=api_key, max_retries=5)  # max_retries 設定最大重試次數

### 計算嵌入（Compute embeddings）

In [ ]:
# 使用 await 等待非同步函式的結果
res = await client.embeddings.create(input=queries[0], model="text-embedding-3-small")
embedding = res.data[0].embedding
print(len(embedding))  # 顯示嵌入向量的維度大小
print(embedding[:5])  # 顯示向量的前5個維度值

In [ ]:
# res.usage.total_tokens 顯示這次 API 呼叫消耗的 token 數量
print(f"Total tokens: {res.usage.total_tokens}")

In [ ]:
# 使用 asyncio.gather 並行處理多個 API 呼叫，以節省時間
_query_embeddings = await asyncio.gather(
    *[
        client.embeddings.create(input=query, model="text-embedding-3-small")
        for query in queries
    ]
)
# 將 API 回傳的嵌入轉換為 PyTorch 張量 (Tensor)
query_embeddings = torch.Tensor(
    [embedding.data[0].embedding for embedding in _query_embeddings]
)

_movie_embeddings = await asyncio.gather(
    *[
        client.embeddings.create(input=movie, model="text-embedding-3-small")
        for movie in movies
    ]
)
movie_embeddings = torch.Tensor(
    [embedding.data[0].embedding for embedding in _movie_embeddings]
)

In [ ]:
# movie_embeddings 已經是 PyTorch 張量了，這裡再次轉換 torch.Tensor(movie_embeddings) 是多餘的
# 但為了保持與 notebook 一致，暫不修改。
torch.Tensor(movie_embeddings)  # 這裡的 movie_embeddings 已經是 torch.Tensor

### 計算相似度並擷取前 K 個（Computing similarity + retrieving top k）

我們使用 `torch.nn.functional.cosine_similarity` (別名為 `F.cosine_similarity`) 來計算餘弦相似度。
`unsqueeze()` 方法用於增加張量的維度，以符合 `cosine_similarity` 的輸入要求。
`dim=2` 表示沿著第三個維度（索引為2）計算相似度。

In [ ]:
similarities = F.cosine_similarity(
    query_embeddings.unsqueeze(1),  # (num_queries, 1, embedding_dim)
    movie_embeddings.unsqueeze(0),  # (1, num_movies, embedding_dim)
    dim=2,  # 結果維度 (num_queries, num_movies)
)
similarities

In [ ]:
YELLOW = "\033[33m"
END = "\033[0m"
for idx_i, sentence1 in enumerate(queries):
    print(sentence1)
    for idx_j in torch.topk(similarities[idx_i], k=4).indices:
        print(f" - {movies[idx_j]: <30}: {YELLOW}{similarities[idx_i][idx_j]:.4f}{END}")

### 計算 token 數量與價格（Calculate tokens and price）

[OpenAI 價格](https://platform.openai.com/docs/pricing)

使用 `tiktoken` 套件可以計算 OpenAI 模型處理文字時會消耗多少 token。
不同的模型有不同的計價方式。

In [ ]:
# 取得 "text-embedding-3-small" 模型對應的編碼器
enc = tiktoken.encoding_for_model("text-embedding-3-small")

In [ ]:
encoded = enc.encode(queries[0])  # 將文字編碼為 token ID 列表
print(f"Total tokens: {len(encoded)}")
print(encoded)

In [ ]:
# 不同嵌入模型的每百萬 token 價格 (美元)
model_to_price = {
    "text-embedding-3-small": 0.02,  # 價格較低
    "text-embedding-3-large": 0.13,  # 價格較高，但通常效果更好
}


def get_token_count_and_price(
    texts: list[str], model: str = "text-embedding-3-small"
) -> tuple[int, float]:
    """
    計算一批文字的總 token 數和預估價格。
    """
    if model not in model_to_price:
        raise ValueError(f"不支援的模型 {model}")
    enc = tiktoken.encoding_for_model(model)
    # encode_batch 可以一次處理多筆文字，效率較高
    token_count = sum(len(e) for e in enc.encode_batch(texts))
    price_per_1m_tokens = model_to_price[model]
    price = (token_count / 1_000_000) * price_per_1m_tokens
    return token_count, price

In [ ]:
# 估算 movies 列表重複10次的 token 數和價格
get_token_count_and_price(movies * 10, model="text-embedding-3-small")

### 處理速率限制（Handling rate limits）

當短時間內大量呼叫 API 時，可能會觸發速率限制 (rate limits)。
`tenacity` 套件可以幫助我們實作重試機制，例如指數退避 (exponential backoff)。
指數退避是指每次重試的等待時間會逐漸增加，避免對 API 造成過大負載。

In [ ]:
def log_backoff_attempt(retry_state: RetryCallState) -> None:
    """
    在每次重試前記錄日誌。
    """
    attempt_num = retry_state.attempt_number  # 第幾次重試
    exception = (
        retry_state.outcome.exception() if retry_state.outcome else "N/A"
    )  # 觸發重試的例外
    wait_time = (
        retry_state.next_action.sleep if retry_state.next_action else 0.0
    )  # 下次重試前的等待時間
    func_name = (
        retry_state.fn.__name__ if retry_state.fn else "N/A"
    )  # 正在重試的函式名稱

    logger.info(
        f"函式 '{func_name}' 退避中： "
        f"第 {attempt_num} 次嘗試因 '{exception.__class__.__name__}: {exception}' 失敗。 "
        f"下次嘗試前等待 {wait_time:.2f} 秒。"
    )


@retry(
    wait=wait_random_exponential(
        min=1, max=60
    ),  # 等待時間為指數增加，最小1秒，最大60秒，並加入隨機性
    stop=stop_after_attempt(6),  # 最多重試6次
    before_sleep=log_backoff_attempt,  # 每次重試前呼叫 log_backoff_attempt
)
async def embedding_with_backoff(**kwargs):
    """
    帶有指數退避重試機制的嵌入函式。
    """
    return await client.embeddings.create(**kwargs)

# 設定向量資料庫（Setting up a vector database）

[LanceDB 文件](https://lancedb.github.io/lancedb/basic/)

向量資料庫專門用於儲存和檢索向量嵌入。
LanceDB 是一個開源的、無伺服器的向量資料庫，很容易在本機端使用。

## 建立 LanceDB 表格（Creating a LanceDB table）

我們將使用電影的 "overview"（概覽）欄位來產生嵌入並存入資料庫。

In [ ]:
# ds 是我們之前預處理好的 Hugging Face Dataset 物件
ds[0]

In [ ]:
overviews = ds["overview"]  # 取出所有電影的概覽文字
print(len(overviews))
overviews[:5]  # 顯示前5筆概覽

In [ ]:
ds.features  # 顯示資料集的欄位資訊和型別

## 使用 sentence-transformers 產生嵌入

我們將使用先前載入的 `sentence-transformers` 模型來為所有電影概覽產生嵌入。
這一步在本機執行，不需要 API 金鑰。

In [ ]:
# 為所有電影概覽產生嵌入
# 這一行程式碼執行時間可能會比較久，取決於資料量和硬體效能
overview_embeddings = embedder.encode(overviews)
overview_embeddings[0][:5]  # 顯示第一個概覽嵌入的前5個維度

In [ ]:
overview_embeddings.shape  # (電影數量, 嵌入維度)

In [ ]:
# 將產生的嵌入向量作為新的一欄 "vector" 加入到我們的資料集中
# .tolist() 是因為 Hugging Face Datasets 在新增欄位時，若資料是 numpy array，通常需要轉成 list
ds = ds.add_column(name="vector", column=overview_embeddings.tolist())

### 將結果儲存到磁碟（Save results to disk）

處理完的資料集可以儲存起來，方便之後載入使用，避免重複運算。
Hugging Face Datasets 提供了 `save_to_disk` 方法。

In [ ]:
ds.save_to_disk("./data/dataset_processed")  # 將資料集儲存到指定路徑

#### 將結果儲存到 Google Drive（Save results to Google Drive）

如果你在 Google Colab 環境中執行，可以將資料儲存到 Google Drive，方便持久化儲存。
這段程式碼只有在 Colab 環境中才需要執行。

In [ ]:
# from google.colab import drive

# drive.mount("/content/drive") # 掛載 Google Drive
# !mkdir -p "/content/drive/My Drive/genai4h-wk14" # 在 Google Drive 建立資料夾
# !cp -r "./data/dataset_processed" "/content/drive/My Drive/genai4h-wk14/" # 將處理好的資料複製到 Google Drive

### 從磁碟載入結果（Load results from disk）

使用 `load_from_disk` 可以快速載入之前儲存的資料集。

In [ ]:
ds = load_from_disk(
    "./data/dataset_processed"
)  # 從磁碟載入處理好的資料集 (注意路徑與儲存時一致)
ds[0]  # 檢查載入的資料

In [ ]:
# 將 Hugging Face Dataset 轉換為 pandas DataFrame，方便後續操作
df = ds.to_pandas()
df.iloc[0]  # 顯示 DataFrame 的第一列

In [ ]:
# 連接到 LanceDB 資料庫
# 如果 "./data/lance_db" 路徑不存在，LanceDB 會自動建立它
db = lancedb.connect("./data/lance_db")

In [ ]:
# 建立一個名為 "movies" 的表格
# data=df 表示使用我們的 DataFrame 作為資料來源
# mode="overwrite" 表示如果 "movies" 表格已存在，則覆寫它
tbl = db.create_table("movies", data=df, mode="overwrite")
# 對 "overview" 欄位建立全文檢索 (FTS) 索引，可以加速關鍵字搜尋
# 注意：這裡的 FTS 索引是針對 "overview" 這個文字欄位，而不是向量欄位
tbl.create_fts_index("overview", replace=True)

### 使用 OpenAI 產生嵌入（OpenAI）

[嵌入太慢？(Slow embeddings?)](https://community.openai.com/t/embeddings-api-extremely-slow/1135044)

如果選擇使用 OpenAI API 來產生所有電影概覽的嵌入，請注意：
1.  **費用**：資料量大時，費用可能會比較高。
2.  **時間**：即使使用 `asyncio.gather` 和重試機制，處理大量文字也可能需要一些時間。

以下程式碼是使用 OpenAI 產生嵌入的範例。如果先前已使用 `sentence-transformers` 產生並儲存了嵌入，可以跳過這部分。

In [ ]:
overviews = ds["overview"]  # 再次取得電影概覽
# 計算使用 OpenAI "text-embedding-3-small" 模型處理所有概覽所需的 token 數和預估價格
get_token_count_and_price(overviews, model="text-embedding-3-small")

In [ ]:
# # 註解掉以下區塊，因為我們已經使用 sentence-transformers 產生嵌入
# # 如果要改用 OpenAI，請取消註解並執行
#
# # 使用帶有重試機制的 embedding_with_backoff 函式，為每個概覽文字產生嵌入
# _overview_embeddings = await asyncio.gather(
#     *[
#         embedding_with_backoff(input=overview, model="text-embedding-3-small")
#         # client.embeddings.create(input=overview, model="text-embedding-3-small") # 不帶重試的版本
#         for overview in overviews
#     ]
# )
# overview_embeddings = [
#     embedding.data[0].embedding for embedding in _overview_embeddings
# ]
# overview_embeddings[0][:5]

In [ ]:
# # 註解掉以下區塊，因為我們已經使用 sentence-transformers 產生嵌入
# df = ds.to_pandas() # 如果前面沒有轉 DataFrame，這裡需要轉換
# df["vector"] = overview_embeddings # 將 OpenAI 產生的嵌入存到 DataFrame 的 "vector" 欄
# # 之後可以像 sentence-transformers 的情況一樣，用這個 df 建立 LanceDB 表格

# 向量搜尋（Vector search）

* [向量搜尋 (Vector search)](https://lancedb.github.io/lancedb/search/)
* [混合搜尋 (Hybrid search)](https://lancedb.github.io/lancedb/hybrid_search/hybrid_search/) (結合向量搜尋和關鍵字搜尋)
* [關鍵字搜尋 (Keyword search)](https://lancedb.github.io/lancedb/fts/) (需要先對文字欄位建立 FTS 索引)

混合搜尋和關鍵字搜尋對於跨語言搜尋的效果可能不如純向量搜尋（如果使用的嵌入模型支援多語言）。
向量搜尋的核心是計算查詢向量與資料庫中所有向量的相似度，並找出最相似的結果。

## 載入嵌入模型（Load embedding model）
<div class="alert alert-block alert-warning">
⚠️ **警告**：必須使用與建立資料庫時相同的嵌入模型來產生查詢向量。否則，向量空間不一致，搜尋結果會沒有意義。
</div>

因為我們之前是用 `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2` 建立的 LanceDB 表格，
所以這裡也要載入同一個模型來產生查詢的嵌入。

In [ ]:
embedder = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device="cpu",
)

In [ ]:
# 連接到先前建立的 LanceDB 資料庫並開啟 "movies" 表格
db = lancedb.connect("./data/lance_db/")
tbl = db.open_table("movies")
# 再次確認 "overview" 欄位有 FTS 索引 (如果之前已建立，replace=True 會覆寫)
# 這一步驟是為了確保 FTS 索引存在，如果只是做向量搜尋，可以不執行。
tbl.create_fts_index("overview", replace=True)

## 嵌入查詢（Embed the query）

將使用者的查詢文字轉換為嵌入向量。

In [ ]:
q_en = "I want to watch a romantic comedy"  # 英文查詢：我想看一部浪漫喜劇
q_zh = "我想看一部浪漫喜劇"  # 中文查詢

q_en_embedding = embedder.encode(q_en)
q_zh_embedding = embedder.encode(q_zh)
print(len(q_en_embedding), len(q_zh_embedding))  # 顯示嵌入向量的維度
print(q_en_embedding[:5], q_zh_embedding[:5])  # 顯示前5個維度值

## 查詢資料庫（Querying the database）

In [ ]:
# 產生一個隨機向量，用於測試
random_vector = torch.randn(10).numpy()  # 產生一個10維的隨機向量
random_vector

<div class="alert alert-block alert-warning">
⚠️ **警告**：以下程式碼會失敗，因為隨機向量的維度 (10) 與資料庫中儲存的嵌入維度 (384，由 `paraphrase-multilingual-MiniLM-L12-v2` 產生) 不符。👇
</div>

In [ ]:
tbl.search(random_vector).limit(5).to_pandas()  # 這行會報錯

In [ ]:
# 使用英文查詢向量進行搜尋
# .select(["overview"]) 指定只回傳 "overview" 欄位
# .limit(5) 指定回傳最相似的5筆結果
# .to_pandas() 將結果轉換為 DataFrame
tbl.search(q_en_embedding).select(["overview"]).limit(5).to_pandas()

In [ ]:
# 使用中文查詢向量進行搜尋
tbl.search(q_zh_embedding).select(["overview", "title"]).limit(5).to_pandas()

# 工具呼叫（Function calling）

工具呼叫（或稱函式呼叫）讓大型語言模型（LLM）可以與外部工具或 API 互動。
LLM 本身不具備執行程式碼或查詢資料庫的能力，但透過工具呼叫，我們可以賦予它這些能力。

我們將以 RAG（檢索增強生成）作為範例，展示 LLM 如何呼叫我們定義的電影資料庫查詢工具。

In [ ]:
# 再次確認嵌入模型和資料庫已載入
embedder = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device="cpu",
)
db = lancedb.connect("./data/lance_db")
tbl = db.open_table("movies")  # 開啟 "movies" 表格

In [ ]:
def query_movie_db(
    text: str,
    limit: int = 10,
) -> ToolCallResult:  # ToolCallResult 是我們自訂的回傳型別
    """
    查詢 LanceDB 電影資料庫，找出與輸入文字概覽相似的電影。

    Args:
        text (str): 用於查詢資料庫的輸入文字。
        limit (int, optional): 回傳結果的數量，預設為 10。

    Returns:
        ToolCallResult: 包含給 LLM 使用的 JSON 字串和給 UI 顯示的 DataFrame。
    """
    q_emb = embedder.encode(text)  # 將查詢文字轉換為嵌入
    df = (
        tbl.search(q_emb)
        .limit(limit)
        .to_pandas()
        .drop(
            columns=["vector", "_distance"]
        )  # 移除 "vector" 和 "_distance" 欄位，簡化輸出
    )
    return {
        "llm_consumable": df.to_json(
            lines=True, orient="records"
        ),  # 轉換為 JSON Lines 格式，方便 LLM 解析
        "ui_displayable": df,  # DataFrame 可以直接在 Jupyter Notebook 中顯示
        "return_type": "dataframe",  # 標註回傳型別
    }

In [ ]:
# 測試 query_movie_db 函式
res = query_movie_db("air bud")  # 查詢關於電影 "air bud" (一隻會打籃球的狗)
print(res["llm_consumable"])  # 印出給 LLM 的 JSON 字串

In [ ]:
res["ui_displayable"].iloc[0]  # 顯示查詢結果 DataFrame 的第一筆資料

## 建立 JSON 結構描述（Creating a JSON schema）

為了讓 LLM 知道我們的工具有哪些參數、以及如何使用它們，我們需要提供一個 JSON 結構描述 (JSON Schema)。
Pydantic BaseModel 可以方便地定義資料結構，並自動產生 JSON Schema。

In [ ]:
# 使用 Pydantic BaseModel 定義 query_movie_db 函式的參數結構
class QueryMovieDB(BaseModel):
    text: str = Field(
        description="用來查詢電影概覽的文字 (Query overviews of movies)",
    )
    limit: int = Field(
        default=10,  # 預設值
        description="回傳結果的數量 (Number of results to return)",
    )

In [ ]:
# create_tool_schema_for_function 是我們在 utils.py 中定義的輔助函式
# 它可以根據函式本身和 Pydantic模型產生符合 LLM 工具呼叫格式的 JSON Schema
schema = create_tool_schema_for_function(query_movie_db, QueryMovieDB)
schema

## Hugging Face InferenceClient

* [Hugging Face InferenceClient 工具呼叫文件](https://huggingface.co/docs/hugs/en/guides/function-calling)

Hugging Face InferenceClient 可以讓我們方便地呼叫部署在 Hugging Face 或其他供應商（如 Fireworks AI）上的 LLM。
有些模型支援工具呼叫功能。

In [ ]:
hf_token = os.getenv("HF_TOKEN")  # 從環境變數讀取 Hugging Face API Token
if hf_token is None and userdata:
    # 如果在 Google Colab 環境，嘗試從 userdata 取得 Hugging Face API Token
    hf_token = userdata.get("HF_TOKEN")
if not hf_token:
    raise ValueError("HF_TOKEN 環境變數未設定")

# 初始化 AsyncInferenceClient
# provider="fireworks-ai" 表示我們將使用 Fireworks AI 提供的模型服務
# 需要有 Fireworks AI 的帳號並設定好 HF_TOKEN (通常與 Fireworks AI 的 API Key 相同或相關)
hf_client = AsyncInferenceClient(
    provider="fireworks-ai",  # 這裡指定使用 Fireworks AI 作為模型提供者
    api_key=hf_token,  # Fireworks AI 的 API 金鑰
)

In [ ]:
messages = [
    {
        "role": "system",
        "content": "不要對數值做假設。如果需要，請要求澄清。(Don't make assumptions about values. Ask for clarification if needed.)",
    },
    {
        "role": "user",
        "content": "我想看一部關於一位被迫重出江湖的退休刺客的電影。 /no_think (I'd like to watch a movie about a retired assassin who is forced back into the game. /no_think)",
        # "/no_think" 是一個提示詞技巧，有時可以讓模型更傾向於呼叫工具而不是直接回答
    },
]

# 呼叫 LLM
# model="Qwen/Qwen3-235B-A22B" 是 Fireworks AI 上的一個模型，支援工具呼叫
# tools=[schema] 提供了我們定義的 query_movie_db 工具的 JSON Schema
# tool_choice="auto" 允許模型自行決定是否以及呼叫哪個工具
# 其他選項: "required": 強制模型呼叫一個或多個工具; "none": 禁止模型呼叫工具
response = await hf_client.chat_completion(
    model="Qwen/Qwen3-235B-A22B",  # 這裡需要填寫 Fireworks AI 上支援工具呼叫的模型 ID
    messages=messages,
    tools=[schema],  # 提供工具的 schema
    tool_choice="auto",
)  # type: ignore # 忽略型別檢查的警告
print(response.choices[0].message.tool_calls)  # 印出模型決定呼叫的工具

In [ ]:
# 完整的模型回覆訊息
response.choices[0].message

In [ ]:
# 模型回覆的工具呼叫中的函式部分 (字串格式)
str(response.choices[0].message.tool_calls[0].function)

In [ ]:
# 解析後的工具呼叫函式物件
func = response.choices[0].message.tool_calls[0].function
func

In [ ]:
print(response.choices[0].message.tool_calls[0].id)  # 工具呼叫的唯一 ID
print(func.name)  # 被呼叫的函式名稱 (應該是 "query_movie_db")
print(json.loads(func.arguments))  # 被呼叫的函式參數 (JSON 字串轉為 Python 字典)

### 串流模式有點麻煩 (Streaming kind of a hassle)

Hugging Face InferenceClient 也支援串流模式 (stream=True)，可以逐步接收模型的輸出。
但在工具呼叫的場景下，解析串流的工具呼叫資訊會比較複雜，需要手動組合訊息片段。

In [ ]:
response = await hf_client.chat_completion(
    model="Qwen/Qwen3-235B-A22B",
    messages=messages,
    tools=[schema],
    tool_choice="auto",
    stream=True,  # 開啟串流模式
)  # type: ignore
chunks = []
async for chunk in response:
    chunks.append(chunk)
    print(chunk)  # 逐塊印出收到的訊息

In [ ]:
messages = [
    {
        "role": "system",
        "content": "你是一個樂於助人的助理。只在你確定需要時才查詢資料庫。(You are a helpful assistant. Only query the database if you are sure it is needed.)",
    },
    {
        "role": "user",
        "content": "我想看一部關於超級英雄的電影。 /no_think (I want to watch a movie about superheroes. /no_think)",
    },
]
# hf_client.chat.completions.create 是另一個呼叫聊天模型的介面，用法類似
response = await hf_client.chat.completions.create(
    model="Qwen/Qwen3-235B-A22B",
    messages=messages,
    tools=[schema],
    tool_choice="auto",
    stream=True,
)  # type: ignore
chunks = []
async for chunk in response:
    chunks.append(chunk)
    print(chunk)

## OpenAI

* [OpenAI 工具呼叫文件](https://platform.openai.com/docs/guides/function-calling?api-mode=chat)
* [慷慨的免費額度 (Generous free tier)](https://platform.openai.com/docs/models/gpt-4.1-nano)

OpenAI 的 API 也支援工具呼叫，並且整合得相當好。
`gpt-4.1-nano` 是一個較新的、可能會有免費額度的模型 (請查閱 OpenAI 最新政策)。

![](https://i.ibb.co/JwZtC9px/Screenshot-2025-05-20-235653.png "GPT-4.1-nano")


In [ ]:
oai_api_key = os.getenv("OPENAI_API_KEY")
if oai_api_key is None and userdata:
    # 如果在 Google Colab 環境，嘗試從 userdata 取得 OpenAI API 金鑰
    oai_api_key = userdata.get("OPENAI_API_KEY")
if not oai_api_key:
    # 如果沒有找到 OpenAI API 金鑰，則拋出錯誤
    raise ValueError("OPENAI_API_KEY 環境變數未設定")
oai_client = AsyncOpenAI(api_key=oai_api_key)  # 初始化 OpenAI 非同步客戶端

In [ ]:
messages = [
    {
        "role": "system",
        "content": "不要對數值做假設。如果需要，請要求澄清。",
    },
    {
        "role": "user",
        "content": "我想看一部關於一位被迫重出江湖的退休刺客的電影。",
    },
]

response = await oai_client.chat.completions.create(
    model="gpt-4.1-nano",  # 或其他支援工具呼叫的 OpenAI 模型
    messages=messages,
    tools=[schema],  # 同樣提供工具的 JSON Schema
    tool_choice="auto",  # 讓模型自動決定
)
print(response.choices[0].message.tool_calls)  # 印出模型回傳的工具呼叫

In [ ]:
# OpenAI 回傳的工具呼叫物件
tool_call = response.choices[0].message.tool_calls[0]
tool_call

In [ ]:
# 建立一個可用的函式字典，方便根據名稱呼叫
AVAILABLE_FUNCTIONS = {
    "query_movie_db": query_movie_db,  # 將函式名稱對應到實際的函式物件
}


def call_function(name: str, args: dict) -> ToolCallResult:
    """
    根據函式名稱和參數呼叫對應的函式。
    包含基本的錯誤處理。
    """
    func = AVAILABLE_FUNCTIONS.get(name)
    if not func:
        # raise ValueError(f"未知的函式： {name}")
        error_msg = f"錯誤：找不到工具 '{name}'。"
        print(error_msg)
        return {
            "llm_consumable": error_msg,
            "ui_displayable": error_msg,
            "return_type": "error_message",
        }
    try:
        # 使用 **args 將字典解包為函式的關鍵字參數
        return func(**args)
    except TypeError as e:  # 捕捉參數不符等 TypeError
        error_msg = f"錯誤：呼叫工具 '{name}' 時參數不符，參數：{args}。詳細資訊：{e}"
        print(error_msg)
        return {
            "llm_consumable": error_msg,
            "ui_displayable": error_msg,
            "return_type": "error_message",
        }
    except Exception as e:  # 捕捉工具執行期間的其他錯誤
        error_msg = f"錯誤：執行工具 '{name}' (參數：{args}) 時發生錯誤。詳細資訊：{e}"
        print(error_msg)
        return {
            "llm_consumable": error_msg,
            "ui_displayable": error_msg,
            "return_type": "error_message",
        }

## 檢查 LLM 回應中的工具呼叫（Check for function calls in LLM response）

當 LLM 決定呼叫工具時，它的回應會包含 `tool_calls` 欄位。
我們需要：
1.  檢查 `tool_calls` 是否存在。
2.  如果存在，解析工具呼叫的資訊（名稱、參數）。
3.  執行對應的工具函式。
4.  將工具函式的執行結果回傳給 LLM，讓它可以根據結果繼續生成回應。

In [ ]:
client = AsyncInferenceClient(provider="fireworks-ai", api_key=hf_token)

messages = [
    {
        "role": "system",
        "content": "Don't make assumptions about values. Ask for clarification if needed.",
    },
    {
        "role": "user",
        "content": "I'd like to watch a movie about a retired assassin who is forced back into the game. /no_think",  # 不要讓模型思考
    },
]

# 呼叫 LLM 的 chat_completion API
# model: 指定要使用的 LLM 模型名稱
# messages: 包含完整對話歷史的訊息列表
# tools: 提供給 LLM 的可用工具的結構描述列表
# tool_choice: "auto" 表示讓 LLM 自行決定是否以及呼叫哪個工具
# stream=True 被註解掉了，表示目前不使用串流模式。串流模式下處理工具呼叫會比較複雜。
response = await client.chat_completion(  # type: ignore # 忽略型別檢查器的警告
    model="Qwen/Qwen3-235B-A22B",  # 使用 Fireworks AI 上的 Qwen 模型
    messages=messages,
    tools=[schema],
    tool_choice="auto",
    # stream=True,  # streaming is a lot of work to handle tool calls and regular messages
)  # type: ignore


# 從 LLM 的回應中取得工具呼叫的資訊
# response.choices[0].message 是 LLM 的主要回覆內容
# .tool_calls 可能包含一個或多個模型決定呼叫的工具
tool_calls = response.choices[0].message.tool_calls

# 檢查 LLM 是否真的要求呼叫工具
if tool_calls:
    # 假設只處理第一個工具呼叫 (如果模型可能一次呼叫多個，這裡需要迴圈處理)
    tc: ChatCompletionOutputToolCall = tool_calls[0]
    # 記錄被呼叫的工具資訊

    # 取得工具呼叫的唯一 ID (call_id)，稍後將工具執行結果傳回給 LLM 時會用到
    call_id = tc.id
    # 取得被呼叫的函式名稱
    func_name = tc.function.name
    # 解析 LLM 提供的函式參數 (從 JSON 字串轉換為 Python 字典)
    func_args = json.loads(tc.function.arguments)

    # 實際執行工具函式
    # call_function 是我們自己定義的函式，它會根據 func_name 和 func_args 執行對應的工具
    tool_result = call_function(func_name, func_args)

# 結構化輸出（Structured outputs）

有時候，我們希望 LLM 的輸出遵循特定的格式，例如 JSON。
結構化輸出可以讓我們更容易地解析和使用 LLM 的回應。

我們可以使用 Pydantic BaseModel 來定義期望的輸出結構。

In [ ]:
# StrEnum 是 Python 3.11+ 的功能，繼承自 str 和 Enum，讓列舉成員本身就是字串
class Polarity(StrEnum):
    POSITIVE = "positive"
    NEGATIVE = "negative"
    NEUTRAL = "neutral"


class SentimentAnalysisOutput(BaseModel):
    # Annotated 可以為欄位加上額外的描述或限制
    polarity: Annotated[Polarity, "文字的情感極性 (The sentiment polarity of the text)"]
    confidence: Annotated[
        float,
        Field(
            description="情感極性的信賴分數，介於 0 和 1 之間 (The confidence score of the sentiment polarity between 0 and 1)",
            ge=0.0,  # ge: greater than or equal to (大於等於)
            le=1.0,  # le: less than or equal to (小於等於)
        ),
    ]


# SentimentAnalysisOutput.model_json_schema() 可以產生此 Pydantic 模型的 JSON Schema
print(json.dumps(SentimentAnalysisOutput.model_json_schema(), indent=2))

In [ ]:
# 這裡會拋出 Pydantic 的 ValidationError，因為 confidence=1.1 超出了定義的範圍 (le=1.0)
try:
    SentimentAnalysisOutput(polarity="positive", confidence=1.1)
except Exception as e:
    print(e)  # 印出錯誤訊息

## 準備提示（Prepare prompt）

提示中需要清楚地告知 LLM 我們期望的 JSON 格式，包含欄位名稱和值的範例。

In [ ]:
base_prompt = """\
請分析以下文字的情感（正面、負面或中性），並以 JSON 格式回傳結果。
JSON 應包含以下欄位：
- polarity: 文字的情感極性（positive、negative 或 neutral）
- confidence: 情感極性的信賴分數，介於 0（不確定）和 1（非常確定）之間
JSON 格式應如下：
{{
    "polarity": "positive",
    "confidence": 0.95
}}
文字： {text}
"""

text_to_analyze = "香菜加在任何東西上都超讚的！(Cilantro is amazing on everything!)"

messages = [
    {
        "role": "user",
        "content": base_prompt.format(text=text_to_analyze),
    }
]

## Hugging Face InferenceClient

對於 Hugging Face InferenceClient，我們需要在 `chat_completion` 的 `response_format` 參數中提供期望的 JSON Schema。
`response_format={"type": "json_object", "value": SCHEMA}`
某些模型（例如較新的 Qwen 模型）支援這個功能。

In [ ]:
response = await hf_client.chat_completion(
    model="Qwen/Qwen3-235B-A22B",  # 確認此模型支援 JSON 模式和 schema 指定
    messages=messages,
    response_format={
        "type": "json_object",  # 指定回傳型別為 JSON 物件
        "value": SentimentAnalysisOutput.model_json_schema(),  # 提供 Pydantic 模型的 JSON Schema
    },
)  # type: ignore
response

In [ ]:
# LLM 回傳的原始 JSON 字串內容
response.choices[0].message.content

In [ ]:
# 將 JSON 字串解析為 Python 字典
response_dict = json.loads(response.choices[0].message.content)
response_dict

In [ ]:
# 使用 Pydantic 模型驗證並轉換字典
# 如果 response_dict 的結構符合 SentimentAnalysisOutput，則會成功轉換
# 否則會拋出 ValidationError
sentiment_result = SentimentAnalysisOutput(**response_dict)
sentiment_result

In [ ]:
# Pydantic 模型提供了 model_validate_json 方法，可以直接從 JSON 字串驗證並轉換
sentiment_result = SentimentAnalysisOutput.model_validate_json(
    response.choices[0].message.content
)
sentiment_result

In [ ]:
# 存取 Pydantic 物件的屬性
str(sentiment_result.polarity)  # .polarity 是 Polarity 列舉型別，str() 可以取得其字串值

## OpenAI

[OpenAI 結構化輸出文件](https://platform.openai.com/docs/guides/structured-outputs?api-mode=chat)

OpenAI 的 Python SDK 提供了更方便的方式來處理結構化輸出。
可以直接將 Pydantic 模型傳遞給 `response_format` 參數 (使用 `openai.beta.chat.completions.parse` 時)。

<div class="alert alert-block alert-warning">
⚠️ **注意**：我們使用 `client.beta.chat.completions.parse` 而不是 `client.chat.completions.create`。
`parse` 方法會自動處理 JSON 解析並驗證 Pydantic 模型。
</div>

In [ ]:
# 使用 client.beta.chat.completions.parse
# 將 Pydantic 模型 SentimentAnalysisOutput 直接傳給 response_format
response = await oai_client.beta.chat.completions.parse(
    messages=messages,
    model="gpt-4.1-nano",  # 或其他支援 JSON 模式的 OpenAI 模型
    response_format=SentimentAnalysisOutput,  # 直接傳遞 Pydantic 模型
)
response  # response 物件本身就包含了 .parsed 的結果

In [ ]:
# LLM 回傳的原始 JSON 字串內容 (如果需要查看)
response.choices[0].message.content

In [ ]:
# 如果需要手動解析 (雖然 parse 方法已經做了)
response_dict = json.loads(response.choices[0].message.content)
response_dict

In [ ]:
# 使用 Pydantic 模型驗證並轉換字典
sentiment_result = SentimentAnalysisOutput(**response_dict)
sentiment_result

In [ ]:
# Pydantic 模型提供了 model_validate_json 方法
sentiment_result = SentimentAnalysisOutput.model_validate_json(
    response.choices[0].message.content
)
sentiment_result

In [ ]:
# 直接存取 .parse() 方法解析好的 Pydantic 物件
# 這是使用 .beta.chat.completions.parse 的主要優點
parsed_output = response.choices[0].message.parsed
print(parsed_output)
print(type(parsed_output))  # 型別應該是 <class '__main__.SentimentAnalysisOutput'>
print(parsed_output.polarity)
print(parsed_output.confidence)

# 聊天機器人（Chatbot）

[Creating a chatbot fast](https://www.gradio.app/guides/creating-a-chatbot-fast)

若要使用 `gr.ChatInterface()` 建立聊天應用程式，您首先應該做的是定義您的聊天函數 。最簡單的情況下，您的聊天函數應接受兩個參數：`message` 和 `history`（參數名稱可以自訂，但必須依照此順序）。

* `message`：一個 `str`，代表使用者最近的訊息。
* `history`：一個 OpenAI 風格的字典列表，其中包含 `role` 和 `content` 鍵，代表先前的對話歷史。也可能包含代表訊息元資料的其他鍵 。

例如，`history` 可能如下所示：

```json
[
  {"role": "user", "content": "What is the capital of France?"},
  {"role": "assistant", "content": "Paris"}
]
```

而下一則 `message` 將會是：

```
"And what is its largest city?"
```

您的聊天函數只需要回傳：

一個 `str` 值，這是聊天機器人根據聊天歷史和最新訊息所做的回應，例如，在這種情況下：

```
Paris is also the largest city.
```

In [ ]:
import gradio as gr


hf_client = AsyncInferenceClient(
    provider="fireworks-ai",
    api_key=hf_token,
)


async def hf_chat(message, history) -> str:
    """
    使用 Hugging Face 的聊天模型進行對話。
    """
    response = await hf_client.chat_completion(
        model="Qwen/Qwen3-235B-A22B",
        messages=history + [{"role": "user", "content": message}],
    )  # type: ignore
    return response.choices[0].message.content


oai_client = AsyncOpenAI(api_key=oai_api_key)


async def oai_chat(message, history) -> str:
    response = await oai_client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=history + [{"role": "user", "content": message}],
    )  # type: ignore
    return response.choices[0].message.content


gr.ChatInterface(
    fn=hf_chat,  # 使用 Hugging Face 的聊天模型
    title="Chat with Qwen",  # 標題
    type="messages",
).launch(share=True)  # 啟動 Gradio 介面，並分享連結

# 測試應用程式
接下來要打開 [`app.py`](app.py)